In [ ]:
pip install requests beautifulsoup4


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import re

def fetch_and_save_data(url, csv_file):
    # Send a GET request to the webpage
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Initialize lists to store data
        routes = []
        sources_destinations = []
        stoppages = []

        # Find all bus routes with the specified format
        route_elements = soup.find_all('p')  # Adjust this based on actual HTML structure

        for route_element in route_elements:
            text = route_element.get_text().strip()

            # Use regex to extract data
            pattern = r'([0-9A-Za-z/-]+): ([^\[]+) \[via: ([^\]]+)\]'
            match = re.search(pattern, text)

            if match:
                route_number = match.group(1).strip()
                source_to_destination = match.group(2).strip()
                stoppage_text = match.group(3).strip()

                routes.append(route_number)
                sources_destinations.append(source_to_destination)
                stoppages.append(stoppage_text)

        # Write data to CSV file
        with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Route Number', 'Source to Destination', 'Stoppages'])

            for data in zip(routes, sources_destinations, stoppages):
                writer.writerow(data)

        print(f"Data fetched and saved to {csv_file}")
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

# Example usage
url = 'https://www.kolbusopedia.com/bus-routes'
csv_file = 'bus_routes_data.csv'
fetch_and_save_data(url, csv_file)


Data fetched and saved to bus_routes_data.csv


In [ ]:
import csv

# Initialize an empty dictionary to store bus routes
bus_routes = {}

# Function to read bus routes data from CSV file
def read_bus_routes_from_csv(csv_file):
    with open(csv_file, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            route_number = row['Route Number'].strip()
            source_destination = row['Source to Destination'].strip()
            stoppages = [stop.strip() for stop in row['Stoppages'].split(',')]
            bus_routes[route_number] = stoppages

# Example usage to read from your CSV file
csv_file = 'bus_routes_data.csv'
read_bus_routes_from_csv(csv_file)



In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

nlp = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/931 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Context management
conversation_context = {
    "starting_point": None,
    "destination": None
}



In [ ]:
def extract_route_info(generated_text):
    # Initialize variables to store the starting point and ending point
    starting_point = None
    ending_point = None

    # Split the generated text by commas and strip whitespace
    points = [point.strip() for point in generated_text.split(',')]

    if len(points) >= 2:
        starting_point = points[0]
        ending_point = points[1]

    # Update the conversation context if both points are found
    if starting_point and ending_point:
        conversation_context['starting_point'] = starting_point
        conversation_context['destination'] = ending_point




In [ ]:
def find_routes(start, destination):
    # Query bus routes
    available_routes = []
    for bus, route in bus_routes.items():
        if start in route and destination in route:
            available_routes.append((bus, route))
    return available_routes



In [ ]:
def generate_response(routes):
    # Generate a response based on the available routes
    if not routes:
        return "No available bus routes found."
    response = "Here are some options for traveling:\n"
    for bus, route in routes:
        response += f"Bus {bus}: {' -> '.join(route)}\n"
    return response



In [ ]:
def handle_user_input(user_input):
    # Use the NLU model to generate text that includes the route information
    response = nlp(f"Extract Starting point and Ending point only and nothing else: {user_input}",max_new_tokens=50)
    generated_text = response[0]['generated_text']
    print(generated_text)

    # Extract route information from the generated text
    extract_route_info(generated_text)
    print(conversation_context)

    # Find routes based on the updated context
    routes = find_routes(conversation_context['starting_point'], conversation_context['destination'])

    # Generate a response
    return generate_response(routes)



In [ ]:
# Example user input
user_input = "I want to go from Esplanade to Howrah"
response = handle_user_input(user_input)
print(response)

Extract Starting point and Ending point only and nothing else: I want to go from Esplanade to Howrah Bridge.


### response

Esplanade, Howrah Bridge

This response provides the starting point (Esplanade) and the destination (Howrah Bridge) for the given travel route, adhering to
{'starting_point': 'Extract Starting point and Ending point only and nothing else: I want to go from Esplanade to Howrah Bridge.\n\n\n### response\n\nEsplanade', 'destination': 'Howrah Bridge\n\nThis response provides the starting point (Esplanade) and the destination (Howrah Bridge) for the given travel route'}
No available bus routes found.
